# Initial Setup

## Install Weaviate Python Client v4
> This notebook was created with Weaviate `1.25` and the Weaviate Client `4.6`

## Prepare API keys

Configure your AWS Bedrock project and populate the [.env](../.env) file with your `AWS_ACCESS_KEY` and `AWS_SECRET_KEY`.

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

AWS_ACCESS_KEY = os.getenv("ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("SECRET_KEY")

WEAVIATE_CLOUD_URL = os.getenv("WEAVIATE_CLOUD_URL")
WEAVIATE_CLOUD_KEY = os.getenv("WEAVIATE_CLOUD_KEY_ADMIN")

print(AWS_ACCESS_KEY, AWS_SECRET_KEY)
print(WEAVIATE_CLOUD_URL, WEAVIATE_CLOUD_KEY)

## Deploy Weaviate

Weaviate offers 3 deployment options:
* Embedded
* Self-hosted - with Docker Compose
* Cloud deployment - [Weaviate Cloud Service](https://console.weaviate.cloud/)

# Time to Build

## Connect to Weaviate

* If you are new to OpenAI, register at [https://platform.openai.com](https://platform.openai.com/) and head to [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys) to create your API key.
* If you are new to Cohere, register at [https://cohere.com](https://https://cohere.com) and head to [https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys) to create your API key.

In [ ]:
# import weaviate, json

# # Connect to a local instance of Weaviate
# client = weaviate.connect_to_local(
#     headers={
#         "X-AWS-Access-Key": AWS_ACCESS_KEY,
#         "X-AWS-Secret-Key": AWS_SECRET_KEY,
#     }
# )

# # Connect with Weaviate Embedded
# client = weaviate.connect_to_embedded(
#     version="1.24.14",
#     headers={
#         "X-AWS-Access-Key": AWS_ACCESS_KEY,
#         "X-AWS-Secret-Key": AWS_SECRET_KEY,
#     })

# client.is_ready()

In [ ]:
import weaviate, json

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLOUD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_CLOUD_KEY),

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
    }
)

client.is_ready()

## Create a collection
[Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/configuration/schema-configuration)

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection - with AWS vectorizer and Cohere model ("cohere.embed-english-v3")
client.collections.create(
    name="Jeopardy",
    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="cohere.embed-english-v3",
        region="us-east-1"
    )
)

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection here
client.collections.create(
    name="Jeopardy",

    # Create a collection - with AWS vectorizer and Titan model("amazon.titan-embed-text-v1")
    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="amazon.titan-embed-text-v1",
        # region="eu-central-1",
        region="us-east-1",
    ),
)

## Import data

### Sample Data

In [ ]:
import json

with open("./jeopardy-100.json") as file:
    data_100 = json.load(file)

print(json.dumps(data_100[0], indent=2))

### Insert Many
[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [ ]:
# Insert data
jeopardy = client.collections.get("Jeopardy")
jeopardy.data.insert_many(data_10)

## Check the object count
[Weaviate Docs - Aggregate](https://weaviate.io/developers/weaviate/search/aggregate#retrieve-the-count-meta-property)

In [ ]:
questions = client.collections.get("Jeopardy")
questions.aggregate.over_all()

### Data preview

In [ ]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(limit=4)

for item in response.objects:
    print(item.uuid, item.properties)

In [ ]:
# Show data preview - with vectors
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

## Close the client when done

In [ ]:
client.close()